In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import requests
import urllib
import json


google_api_key = '삭제했음'
kk_api_key = '삭제했음'

In [8]:
pip install ahpy

In [11]:
# 현위치 위도/경도 구하기
# url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={google_api_key}'
# data = {
#     'considerIp': True,
# }

# current_location = requests.post(url, data)
# lat = current_location.json()['location']['lat']
# lng = current_location.json()['location']['lng']

# 위도/경도 마포구로 고정
lat = 37.5466701
lng = 126.9244874

dessert_list = ['마카롱', '티라미수', '크로플', '빙수', '소금빵', '푸딩', '젤라또', '츄러스', '케이크', '개성주악',
                '에클레어', '크루키', '휘낭시에','모찌', '와플', '베이글', '퀸아망', '떡볶이', '핫도그', '약과', '도넛'
                '떡', '커피', '탕후루', '호떡', '꽈배기', '까눌레', '크림빵', '쿠키', '머핀', '양갱'] ## 추가 가능

def keyword_dessert(keyword):
    url_loc= 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {'Authorization': f'KakaoAK {kk_api_key}'}

    params = {
        'query':keyword,       # 검색 키워드
        'x' :lng,
        'y' :lat,
        'radius': 2000          # 반경 2km 이내 검색
    }

    response = requests.get(url_loc, headers=headers, params=params)
    key_dessert, dist_dessert = [], []

    places = response.json()
    for place in places['documents']:
        key_dessert.append(place['place_name'])
        dist_dessert.append(place['distance'])

    return key_dessert, dist_dessert

key, dist, d_type = [], [], []
for d in dessert_list:
    dessert = d
    t1, t2 = keyword_dessert(dessert)
    key.extend(t1)
    dist.extend(t2)
    d_type.extend([d]*len(t1))

In [12]:
# 가게별 평점 구성
def store_ratings(name):
    enc_name = urllib.parse.quote(name)
    default_place_id = "ChIJVVX5m2aLGGARdMqGzYFGs9k"

    url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json' \
    f'?fields=name%2Cformatted_address%2Crating%2Cplace_id' \
    f'&input={enc_name}&inputtype=textquery&key={google_api_key}'

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload).json()
    if response.get('status') == 'ZERO_RESULTS':
        place_id = default_place_id #사전에 지정한 가게의 id를 디폴트로
    else:
        place_id = response['candidates'][0]['place_id']

    url_detailed = "https://maps.googleapis.com/maps/api/place/details/json"\
    "?fields=name%2Crating%2Ctakeout%2Cprice_level%2Cuser_ratings_total%2Cformatted_phone_number%2Copening_hours"\
    f"&place_id={place_id}"\
    f'&inputtype=textquery&key={google_api_key}'

    response_detailed = requests.request("GET", url_detailed, headers=headers, data=payload).json()
    return response_detailed['result']
    # if 'result' not in response_detailed:
    #     print('error', name)
    # else:
    #     pass

In [13]:
store_ratings('설빙 홍대입구역점')

{'formatted_phone_number': '03-5778-6161',
 'name': 'UN GRAIN',
 'opening_hours': {'open_now': False,
  'periods': [{'close': {'day': 0, 'time': '1800'},
    'open': {'day': 0, 'time': '1100'}},
   {'close': {'day': 1, 'time': '1800'}, 'open': {'day': 1, 'time': '1100'}},
   {'close': {'day': 2, 'time': '1800'}, 'open': {'day': 2, 'time': '1100'}},
   {'close': {'day': 4, 'time': '1800'}, 'open': {'day': 4, 'time': '1100'}},
   {'close': {'day': 5, 'time': '1800'}, 'open': {'day': 5, 'time': '1100'}},
   {'close': {'day': 6, 'time': '1800'}, 'open': {'day': 6, 'time': '1100'}}],
  'weekday_text': ['Monday: 11:00\u202fAM\u2009–\u20096:00\u202fPM',
   'Tuesday: 11:00\u202fAM\u2009–\u20096:00\u202fPM',
   'Wednesday: Closed',
   'Thursday: 11:00\u202fAM\u2009–\u20096:00\u202fPM',
   'Friday: 11:00\u202fAM\u2009–\u20096:00\u202fPM',
   'Saturday: 11:00\u202fAM\u2009–\u20096:00\u202fPM',
   'Sunday: 11:00\u202fAM\u2009–\u20096:00\u202fPM']},
 'price_level': 3,
 'rating': 4.4,
 'takeout': Tr

In [14]:
for name in key:
    store_ratings(name) #수정) 문제 없음

In [15]:
df = pd.DataFrame({
    '종류': d_type,
    '가게명_한글': key,
    '거리': dist
})

to_drop = []
for i in range(len(df)):
    store_rate = store_ratings(df.iloc[i]['가게명_한글'])
    if store_rate.get('name') != 'UN GRAIN':
        df.at[i, '가게명_영문'] = store_rate.get('name', np.nan)
        df.at[i, '전화번호'] = store_rate.get('formatted_phone_number', np.nan)
        df.at[i, '전체 리뷰 수'] = store_rate.get('user_ratings_total', np.nan)
        df.at[i, '구글맵 평점'] = store_rate.get('rating', np.nan)
        df.at[i, '가격대'] = store_rate.get('price_level', np.nan)
        df.at[i, '포장 가능'] = store_rate.get('takeout', np.nan)
        df.at[i, '영업중'] = store_rate.get('opening_hours', {}).get('open_now', np.nan)
    else:
        to_drop.append(i)

df.drop(to_drop, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

<ipython-input-15-f90de35602c3>:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, '포장 가능'] = store_rate.get('takeout', np.nan)
<ipython-input-15-f90de35602c3>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, '영업중'] = store_rate.get('opening_hours', {}).get('open_now', np.nan)


In [16]:
df

,종류,가게명_한글,거리,가게명_영문,전화번호,전체 리뷰 수,구글맵 평점,가격대,포장 가능,영업중
0,마카롱,달콤한거짓말,377,달콤한 거짓말,02-3141-3338,115.0,4.4,NaN,True,True
1,마카롱,민트하임,699,Mint Heim,02-324-1359,323.0,4.2,NaN,True,True
2,마카롱,봉쥬르마카롱,1117,봉쥬르마카롱,010-4688-2658,2.0,4.0,NaN,True,False
3,마카롱,유니마카롱 홍대점,289,유니크디저트,02-338-4265,3.0,5.0,NaN,True,True
4,마카롱,플랑,287,플랑플랑,02-542-1020,66.0,4.5,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...
327,양갱,금옥당 서교점,436,금옥당 서교점,02-325-4040,211.0,4.0,NaN,True,True
328,양갱,양갱상점,436,모던양갱집,010-2740-0173,11.0,4.6,NaN,True,True
329,양갱,무제서울,505,Muje,NaN,3.0,4.7,NaN,False,True
330,양갱,트릴로지,1462,Trilogy Restaurant,(919) 357-9887,79.0,3.9,2.0,True,False


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   종류       332 non-null    object 
 1   가게명_한글   332 non-null    object 
 2   거리       332 non-null    object 
 3   가게명_영문   332 non-null    object 
 4   전화번호     244 non-null    object 
 5   전체 리뷰 수  293 non-null    float64
 6   구글맵 평점   293 non-null    float64
 7   가격대      61 non-null     float64
 8   포장 가능    240 non-null    object 
 9   영업중      267 non-null    object 
dtypes: float64(3), object(7)
memory usage: 26.1+ KB


In [18]:
# 포장 가능/영업중(T/F) 1/0 변환
# nan 값을 어떻게 처리할 것인지?

df2 = df.dropna(subset=['포장 가능', '영업중', '전체 리뷰 수', '구글맵 평점'])
df2.reset_index(drop=True, inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   종류       209 non-null    object 
 1   가게명_한글   209 non-null    object 
 2   거리       209 non-null    object 
 3   가게명_영문   209 non-null    object 
 4   전화번호     177 non-null    object 
 5   전체 리뷰 수  209 non-null    float64
 6   구글맵 평점   209 non-null    float64
 7   가격대      58 non-null     float64
 8   포장 가능    209 non-null    object 
 9   영업중      209 non-null    object 
dtypes: float64(3), object(7)
memory usage: 16.5+ KB


In [19]:
df2

,종류,가게명_한글,거리,가게명_영문,전화번호,전체 리뷰 수,구글맵 평점,가격대,포장 가능,영업중
0,마카롱,달콤한거짓말,377,달콤한 거짓말,02-3141-3338,115.0,4.4,NaN,True,True
1,마카롱,민트하임,699,Mint Heim,02-324-1359,323.0,4.2,NaN,True,True
2,마카롱,봉쥬르마카롱,1117,봉쥬르마카롱,010-4688-2658,2.0,4.0,NaN,True,False
3,마카롱,유니마카롱 홍대점,289,유니크디저트,02-338-4265,3.0,5.0,NaN,True,True
4,마카롱,빌리프커피로스터스,1119,Belief Coffee Roasters,02-333-3100,732.0,4.3,2.0,True,True
...,...,...,...,...,...,...,...,...,...,...
204,머핀,에이프릴라임,1783,Happy Ice Cream,NaN,107.0,4.8,2.0,True,False
205,양갱,금옥당 서교점,436,금옥당 서교점,02-325-4040,211.0,4.0,NaN,True,True
206,양갱,양갱상점,436,모던양갱집,010-2740-0173,11.0,4.6,NaN,True,True
207,양갱,무제서울,505,Muje,NaN,3.0,4.7,NaN,False,True


--------

In [9]:
import ahpy

# 쌍대 비교 행렬 데이터 정의
comparisons = {
    ('영업중', '포장 가능'): 5,
    ('영업중', '거리'): 3,
    ('영업중', '전체 리뷰 수'): 7,
    ('영업중', '구글맵 평점'): 6,
    ('영업중', '리뷰 폴라리티'): 6,
    ('영업중', 'Sentiment Score'): 5,
    ('포장 가능', '거리'): 1/3,
    ('포장 가능', '전체 리뷰 수'): 1/5,
    ('포장 가능', '구글맵 평점'): 1/4,
    ('포장 가능', '리뷰 폴라리티'): 1/4,
    ('포장 가능', 'Sentiment Score'): 1/3,
    ('거리', '전체 리뷰 수'): 1/2,
    ('거리', '구글맵 평점'): 1/3,
    ('거리', '리뷰 폴라리티'): 1/3,
    ('거리', 'Sentiment Score'): 1/2,
    ('전체 리뷰 수', '구글맵 평점'): 3,
    ('전체 리뷰 수', '리뷰 폴라리티'): 2,
    ('전체 리뷰 수', 'Sentiment Score'): 2,
    ('구글맵 평점', '리뷰 폴라리티'): 1,
    ('구글맵 평점', 'Sentiment Score'): 1,
    ('리뷰 폴라리티', 'Sentiment Score'): 1
}

# AHP 모델 생성
ahp_model = ahpy.Compare(name='Dessert Criteria', comparisons=comparisons, precision=3)

# 결과 출력 (가중치)
print("가중치:")
print(ahp_model.target_weights)

# 일관성 비율 출력
print(f"Consistency Ratio (CR): {ahp_model.consistency_ratio}")

가중치:
{'영업중': 0.46, '전체 리뷰 수': 0.155, '리뷰 폴라리티': 0.1, '구글맵 평점': 0.097, 'Sentiment Score': 0.089, '거리': 0.066, '포장 가능': 0.033}
Consistency Ratio (CR): 0.093


In [25]:
## 수정한 가중치 계산


dt1 = pd.read_csv('/content/drive/MyDrive/2024 서울특별시 빅데이터 캠퍼스 공모전/분석결과 및 데이터/df2.csv')
dt1 = dt1.drop_duplicates(subset='가게명_한글')
dt1.reset_index(inplace=True, drop=True)
dt1['영업중'] = dt1['영업중'].apply(lambda x:1 if x== True else 0)
dt1['포장 가능'] = dt1['포장 가능'].apply(lambda x:1 if x== True else 0)
dt1 = dt1.drop(['가게명_영문', '전화번호', '가격대'], axis=1)

dt2 = pd.read_csv('/content/drive/MyDrive/2024 서울특별시 빅데이터 캠퍼스 공모전/분석결과 및 데이터/sorted_data.csv')
dt2 = dt2[['가게명_한글', 'Sentiment Score']]

dessert_kind = input()
df = pd.merge(dt1, dt2, how='right', on='가게명_한글')


df = df.loc[df['종류'] ==dessert_kind] # 수정 구간: 단, 5개 이상 결과가 나오는 키워드 입력해야 함

scaler = MinMaxScaler()
columns_to_normalize = ['거리', '전체 리뷰 수', '구글맵 평점', '리뷰 폴라리티', 'Sentiment Score']
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

df['거리'] = 1 - df['거리']

# 가중치 설정
weights = {
    '영업중': 0.46,
    '포장 가능': 0.033,
    '거리': 0.066,
    '전체 리뷰 수': 0.155,
    '구글맵 평점': 0.097,
    '리뷰 폴라리티': 0.1,
    'Sentiment Score': 0.089
}

# 가중치 계산
df['총점'] = (
    # df['종류'] * weights['종류'] +
    df['영업중'] * weights['영업중'] +
    df['포장 가능'] * weights['포장 가능'] +
    df['거리'] * weights['거리'] +
    df['전체 리뷰 수'] * weights['전체 리뷰 수'] +
    df['구글맵 평점'] * weights['구글맵 평점'] +
    df['리뷰 폴라리티'] * weights['리뷰 폴라리티'] +
    df['Sentiment Score'] * weights['Sentiment Score']
)

# 점수가 높은 상위 5개 가게 선정
top_5_stores = df.nlargest(5, '총점')

# 결과 출력
print(top_5_stores[['가게명_한글', '총점']])

베이글
        가게명_한글        총점
27    뉴욕베이글애비뉴  0.952160
168  베이글카페 홍대점  0.888212
49    말통커피앤베이글  0.764700
97       쉬즈베이글  0.752318
22   쉬즈베이글 홍대점  0.731297
